In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
#Download data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

testing_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:43<00:00, 611418.92it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 204276.59it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:02<00:00, 1846723.93it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

for x, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {x.shape}")
    print(f"Shape of Y: {y.shape} {y.dtype}")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape o

### Device

In [4]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_attack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_attack(x)
        return logits

Using cpu


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_attack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def trainer(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        #Compute prediction error
        pred = model(x)
        loss = loss_fn(pred, y)

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"Loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def tester(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f}")

In [9]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1} \n-------------------------------------------------")
    trainer(train_dataloader, model, loss_fn, optimizer)
    tester(test_dataloader, model, loss_fn)


Epoch 1 
-------------------------------------------------


Loss: 2.295624  [   64/60000]
Loss: 2.288453  [ 6464/60000]
Loss: 2.273321  [12864/60000]
Loss: 2.270171  [19264/60000]
Loss: 2.265404  [25664/60000]
Loss: 2.226030  [32064/60000]
Loss: 2.241646  [38464/60000]
Loss: 2.204558  [44864/60000]
Loss: 2.207329  [51264/60000]
Loss: 2.176210  [57664/60000]
Test Error: 
 Accuracy: 32.3%, Avg loss: 2.171661
Epoch 2 
-------------------------------------------------
Loss: 2.181251  [   64/60000]
Loss: 2.175766  [ 6464/60000]
Loss: 2.126353  [12864/60000]
Loss: 2.136767  [19264/60000]
Loss: 2.111105  [25664/60000]
Loss: 2.040531  [32064/60000]
Loss: 2.074312  [38464/60000]
Loss: 1.999888  [44864/60000]
Loss: 2.009289  [51264/60000]
Loss: 1.938798  [57664/60000]
Test Error: 
 Accuracy: 57.5%, Avg loss: 1.937944
Epoch 3 
-------------------------------------------------
Loss: 1.970304  [   64/60000]
Loss: 1.942972  [ 6464/60000]
Loss: 1.837373  [12864/60000]
Loss: 1.865212  [19264/60000]
Loss: 1.776497  [25664/60000]
Loss: 1.712606  [32064/60000]
Lo

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved")

Saved


In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))